---
Finalmente estamos prontos para fazer o sistema de recomendação. Vamos carregar os dados produzidos no notebook, e utilizar cosine-similarity para checar a similaridade dos titúlos. Para isto precisamos importar algumas bibliotecas, a saber : Numpy e sklearn.
---


Importando Bibliotecas e os dados

In [131]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

url = "https://raw.githubusercontent.com/arthurtorres/Projetos/master/Netflix/data_informacao/data.csv"
data = pd.read_csv(url,sep=",")




Vamos então aplicar cosine similarity na coluna objetivo do nosso dataset. Isto criara uma matrix com todos as similaridades entre os titulos, isto é, uma matrix 6234 * 6234

In [132]:

cv = CountVectorizer() #creating new CountVectorizer() object
count_matrix = cv.fit_transform(data["objetivo"]) #feeding 
cosines = cosine_similarity(count_matrix,count_matrix)
cosines.shape

(6234, 6234)

Depois de criado a matrix de similaridade, precisamos tratar do input que receberemos. Receberemos varios titulos, separados por virgula, precisamos então checar se este titulo esta presente e caso positivo, checar suas similaridade. Vamos criar uma função para isto.

In [133]:

def checkaseries(series) :
 all_titles = [data['Titulo'][i].lower() for i in range(len(data['Titulo']))]

 obrasList = []
 for serie in series :
  serie = serie.lower()
  if (serie in all_titles) :
   obrasList.append(serie)
 return obrasList


Vamos testar esta função com Hilda e Vikings. Hilda esta no catalogo da netflix e Vikings não esta, então a função deveria nos retornar apenas Hilda.

In [134]:
checkaseries(["Hilda","Vikings"])


['hilda']

A função parece estar funcionando como deveria. Vamos então criar uma função que retorna series similares e o indice da serie procurada, pois precisamos retirar a serie procurada pois a similaridade de uma serie com ela mesma é 1, e por isto ela sempre sera recomenda primeiro. Para uma lista de series, precisamos guardar o indice referente a cada uma delas.
 

In [135]:
def seriesSimilares(listaObra):

  """
  A partir de uma Lista de obras, retorna o indice dessa serie e seu vetor de series similares.
  """
  recomendacoes = []
  indexes = [] 
  data["Titulo"] = data["Titulo"].str.lower()
  for obra in listaObra :
    obra = obra.lower()
    serie_index = data[data.Titulo == obra].index[0]
    similar_movies = list((cosines[serie_index]))
    recomendacoes.append(similar_movies)
    indexes.append(serie_index)

  return recomendacoes,indexes

Vamos testar a função com hilda e ver o que ela nos retorna.

In [136]:
recom, ind = seriesSimilares(["Hilda"])
recom[0][:5]

[0.0, 0.0, 0.2100420126042015, 0.25000000000000006, 0.0]

Essa lista nos mostra que os 5 primeiros indices tem um valor baixo de similaridade com o valor pesquisado, neste caso, Hilda.

Precisamos então, organizar a saida pelos maiores valores, pois indicará uma similaridade maior com o(s) iten(s) pesquisados. Precisamos também retirar os indices dos iten(s) pesquisados para não afetar os dados. Faremos isto numa função.

In [137]:

def relevancia(similares,indices) :
  """
  Remove os indices e a lista de similaridade e organiza por ordem de Relevancia
  Retorna uma lista
  """

  relevancia =list((enumerate((np.mean(similares,axis =0 )))))
              
  for index in sorted(indices, reverse=True):
      del relevancia[index]

  return    sorted(relevancia,key=lambda x:x[1],reverse=True)



Vamos testar com Hilda.

In [138]:
relevancia(recom, ind)[:5]


[(307, 0.3872983346207417),
 (6061, 0.36084391824351614),
 (1218, 0.3273268353539886),
 (375, 0.3202563076101743),
 (180, 0.2886751345948129)]

Temos então que os filmes mais similares com Hilda são os de indice 307,6061,1218,375 e 180. Precisamos então apenas identificar estes indices no nosso dataset inicial e mostra-los. Faremos isto

In [139]:

fin = relevancia(recom, ind)

r3 = [i[0] for i in fin]
r3 = r3[:6]
names = []
res = datas.loc[r3] 
res = res[["Titulo"]]	
for i in range(len(res)):
 print(res.iloc[i][0])


Loo Loo Kids: Johny & Friends Musical Adventures
The Garfield Show
Brainchild
LEGO Elves: Secrets of Elvendale
Transformers: Rescue Bots
The Mr. Peabody and Sherman Show


Temos que as atrações mais simialres a Hilda em cima. Vamos criar uma função que recebe uma lista separada por vírgulas e retorna as 5 atrações mais similares, isto é as recomendações.

In [140]:
def recomendacoes(series ) :
  cv = CountVectorizer() #creating new CountVectorizer() object
  count_matrix = cv.fit_transform(data["objetivo"]) #feeding 
  cosines = cosine_similarity(count_matrix,count_matrix)
  serie = series.split(",")
  Obra = checkaseries(serie)
  recom, ind = seriesSimilares(Obra)
  fin = relevancia(recom, ind)

  r3 = [i[0] for i in fin]
  r3 = r3[:6]
  res = datas.loc[r3] 
  res = res[["Titulo"]]	
  print(f"As obras recomendadas sa partir da seleção :{series} é \n")
  for i in range(len(res)):
   print(res.iloc[i][0])

Teste da função :

In [141]:
series = "Hilda,Vikings,Umbrella Academy"
recomendacoes(series)

As obras recomendadas sa partir da seleção :Hilda,Vikings,Umbrella Academy é 

Loo Loo Kids: Johny & Friends Musical Adventures
The Garfield Show
Brainchild
LEGO Elves: Secrets of Elvendale
Transformers: Rescue Bots
The Mr. Peabody and Sherman Show


O codigo parece estar funcionando como planejado. O proximo passo seria fazer um site para hospedar esse sistema e obter um sistema de feedback para aprimorar as recomendações.